In [1]:
import pandas as pd
import sys
sys.path.append("..")
from src.query_funcs import *

## Formateo de df para las distintas tablas de la BBDD

In [2]:
df = pd.read_csv("../datos/df_complete.csv", index_col=0).fillna('Girasol')

Empezamos con la tabla de categorías, la cual solo tendrá el id (PK) de la categoría y su nombre. En este generamos los PKs desde python, sin tener que depender de la generación serial de SQL.

In [3]:
cat_enum = enumerate(df["Categoria"].unique(), start=1)
cat_table =list(cat_enum)
cat_enum = enumerate(df["Categoria"].unique(), start=1)
cat_dict = dict(cat_enum)

Los valores a introducir serán entonces:

In [4]:
cat_table

[(1, 'Aceite de girasol'), (2, 'Aceite de oliva'), (3, 'Leche')]

Creamos un mapper que nos permita asociar cada valor a su id.

In [5]:
cat_mapper = dict()
for k,v in cat_dict.items():
    cat_mapper[v] = k
cat_mapper

{'Aceite de girasol': 1, 'Aceite de oliva': 2, 'Leche': 3}

Ahora continuamos con subcategorías, la cual contendrá una columna con el id (PK), otra con el nombre, y una con el id de su categoría correspondiente (FK).

In [6]:
subcat_enum = enumerate(df["Subcategoria"].unique(), start=1)
subcat_dict = dict(subcat_enum)
subcat_dict

{1: 'Girasol',
 2: 'Suave e intenso',
 3: 'Virgen',
 4: 'Virgen extra',
 5: 'Enriquecida',
 6: 'Entera, semi o desnatada',
 7: 'Sin lactosa'}

In [7]:
subcat_cat_df = df[["Subcategoria","Categoria"]].drop_duplicates()
subcat_cat_df["Categoria"] = subcat_cat_df["Categoria"].map(cat_mapper)
subcat_cat_df

,Subcategoria,Categoria
0,Girasol,1
214,Suave e intenso,2
468,Virgen,2
722,Virgen extra,2
1351,Enriquecida,3
1870,"Entera, semi o desnatada",3
3412,Sin lactosa,3


Así nos quedaría la lista de tuplas a introducir.

In [8]:
subcat_table = []
for i in range(len(df["Subcategoria"].unique())):
    subcat_table.append(tuple([i+1,subcat_cat_df.values[i][0], subcat_cat_df.values[i][1]]))
subcat_table

[(1, 'Girasol', 1),
 (2, 'Suave e intenso', 2),
 (3, 'Virgen', 2),
 (4, 'Virgen extra', 2),
 (5, 'Enriquecida', 3),
 (6, 'Entera, semi o desnatada', 3),
 (7, 'Sin lactosa', 3)]

Y el mapper para futuro uso.

In [9]:
subcat_mapper = dict()
for i in subcat_table:
    subcat_mapper[i[1]] = i[0]

subcat_mapper

{'Girasol': 1,
 'Suave e intenso': 2,
 'Virgen': 3,
 'Virgen extra': 4,
 'Enriquecida': 5,
 'Entera, semi o desnatada': 6,
 'Sin lactosa': 7}

Continuamos con los supermercados. Esta tabla es como la de categorías, id (PK) y nombre del supermercado.

In [10]:
super_table = list(enumerate(df["Supermercado"].unique(), start=1))
super_table

[(1, 'mercadona'),
 (2, 'carrefour'),
 (3, 'eroski'),
 (4, 'dia'),
 (5, 'hipercor'),
 (6, 'alcampo')]

Y de nuevo un mapper.

In [11]:
super_dict = dict(super_table)
super_mapper = dict()
for k,v in super_dict.items():
    super_mapper[v] = k
    
super_mapper

{'mercadona': 1,
 'carrefour': 2,
 'eroski': 3,
 'dia': 4,
 'hipercor': 5,
 'alcampo': 6}

Seguimos con los productos. Esta tabla consistirá en un id de producto (PK) generado en el script, el nombre del producto, la subcategoría (FK) y el supermercado (FK). No añadimos categoría porque ya está asociada con subcategoría, e incluimos supermercado aquí y no en la tabla de registros ya que al comprobar los duplicados, no existen artículos compartidos entre supermercados (al menos viendo el nombre). Puede que haya productos compartidos, pero cada supermercado lo llama de forma distinta.

In [12]:
df[["Nombre", "Subcategoria", "Supermercado"]].drop_duplicates().reset_index(drop = True).reset_index().head()

,index,Nombre,Subcategoria,Supermercado
0,0,"Aceite de girasol refinado 0,2º Hacendado 1 l....",Girasol,mercadona
1,1,"Aceite de girasol refinado 0,2º Hacendado 5 l....",Girasol,mercadona
2,2,Aceite de oliva intenso Hacendado 3 l. en Merc...,Suave e intenso,mercadona
3,3,Aceite de oliva suave Hacendado 3 l. en Mercadona,Suave e intenso,mercadona
4,4,Aceite de oliva virgen Hacendado 1 l. en Merca...,Virgen,mercadona


In [13]:
productos_df = df[["Nombre", "Subcategoria", "Supermercado"]].drop_duplicates().reset_index(drop = True).reset_index()
productos_df["Subcategoria"] = productos_df["Subcategoria"].map(subcat_mapper)
productos_df["Supermercado"] = productos_df["Supermercado"].map(super_mapper)
productos_df["index"] = productos_df["index"].map(lambda x:x+1)

In [14]:
productos_table = [tuple(prod) for prod in productos_df.values]

La lista de tuplas quedaría como:

In [15]:
productos_table[:5]

[(1, 'Aceite de girasol refinado 0,2º Hacendado 1 l. en Mercadona', 1, 1),
 (2, 'Aceite de girasol refinado 0,2º Hacendado 5 l. en Mercadona', 1, 1),
 (3, 'Aceite de oliva intenso Hacendado 3 l. en Mercadona', 2, 1),
 (4, 'Aceite de oliva suave Hacendado 3 l. en Mercadona', 2, 1),
 (5, 'Aceite de oliva virgen Hacendado 1 l. en Mercadona', 3, 1)]

Creamos otro mapper para los productos.

In [16]:
prod_mapper = dict(productos_df[["Nombre","index"]].values)

Por último, la tabla de registros. Esta contendrá el id de registro (PK), el id de producto (FK), el precio, la variación absoluta y la variación percentual.

In [17]:
df_registros = df[["Día","Nombre", "Precio (€)", "abs_variacion", "precent_variacion"]].reset_index()
df_registros["Nombre"] = df_registros["Nombre"].map(prod_mapper)
df_registros["index"] = df_registros["index"].map(lambda x:x+1)
registros_table = [tuple(r) for r in df_registros.values]
registros_table[:5]

[(1, '2024-07-12', 1, 1.45, 0.0, 0.0),
 (2, '2024-07-13', 1, 1.45, 0.0, 0.0),
 (3, '2024-07-14', 1, 1.45, 0.0, 0.0),
 (4, '2024-07-15', 1, 1.45, 0.0, 0.0),
 (5, '2024-07-16', 1, 1.45, 0.0, 0.0)]

## Creación de Tablas

In [18]:
query = '''
CREATE TABLE if not exists supermercado(
        id_supermercado int primary key,
        nombre varchar(100) not null);

CREATE TABLE if not exists categorias(
        id_categoria int primary key,
        nombre varchar(100) not null);

CREATE TABLE if not exists subcategorias(
        id_subcat int primary key,
        nombre varchar(100) not null,
        id_categoria int not null,
        foreign key (id_categoria)
                references categorias(id_categoria)
                on delete restrict
                on update cascade);

CREATE TABLE if not exists productos(
        id_producto int primary key,
        nombre varchar(1000) not null,
        id_subcat int not null,
        id_supermercado int not null,
        foreign key (id_subcat)
                references subcategorias(id_subcat)
                on delete restrict
                on update cascade,
        foreign key (id_supermercado)
                references supermercado(id_supermercado)
                on delete restrict
                on update cascade);

CREATE TABLE if not exists registros(
        id_registro int primary key,
        fecha date not null,
        id_producto int not null,
        precio decimal(10,2),
        var_abs decimal(10,2),
        var_porc decimal(10,2),
        foreign key (id_producto)
                references productos(id_producto)
                on delete restrict
                on update cascade);        
        
        '''

conexion = conectar()
query_commit(conexion, query_text=query)

Done!


In [19]:
conexion = conectar()
query_commit_many(conexion, '''insert into supermercado
                                values (%s,%s)''', super_table)

Done!


In [20]:
conexion = conectar()
query_commit_many(conexion, '''insert into categorias
                                values (%s,%s)''', cat_table)

Done!


In [21]:
conexion = conectar()
query_commit_many(conexion, '''insert into subcategorias
                                values (%s,%s,%s)''', subcat_table)

Done!


In [22]:
conexion = conectar()
query_commit_many(conexion, '''insert into productos
                                values (%s,%s,%s,%s)''', productos_table)

Done!


In [23]:
conexion = conectar()
query_commit_many(conexion, '''insert into registros
                                values (%s,%s,%s,%s,%s,%s)''', registros_table)

Done!
